In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook
from tqdm.notebook import tqdm_notebook
import time

In [2]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Load API Credentials
with open('/Users/deidrehunt/.secret/yelp_api.json', 'r') as f:
    login = json.load(f)

# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
login.keys()


dict_keys(['client-id', 'api-key'])

In [4]:
#set our API call parameters and file name before first call
location = 'Washington, DC 20057'
term = 'pizza'


In [5]:
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

Pizza places in Washington, DC

In [6]:
location.split(',')[0]

'Washington'

In [7]:
#Specify folder for saving data
FOLDER = 'Data/'
os.makedirs(FOLDER, exist_ok = True)
JSON_FILE = FOLDER+f"{location.split(',') [0]}-{term}.json"

In [8]:
JSON_FILE

'Data/Washington-pizza.json'

In [9]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/Washington-pizza.json already exists.


Making json calls

In [10]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')


- 0 previous results found.


In [11]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location='Washington, DC 20057',
                                term='pizza')
results.keys()


dict_keys(['businesses', 'total', 'region'])

In [12]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page


20

In [13]:
## How many results total?
total_results = results['total']
total_results


32

In [14]:
total_results = results['region']
total_results

{'center': {'longitude': -77.07557201385498, 'latitude': 38.909268231541304}}

In [16]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages


2

In [21]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location= 'Washington, DC 20057',
                                    term='pizza', 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)


  0%|          | 0/2 [00:00<?, ?it/s]

Open the Final JSON File with Pandas

In [22]:
df = pd.read_json(JSON_FILE)

In [23]:
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,ZpRRIz2PdzzDNOi9mgwkTg,pizzeria-paradiso-georgetown-washington-2,Pizzeria Paradiso Georgetown,https://s3-media2.fl.yelpcdn.com/bphoto/xhLgxG...,False,https://www.yelp.com/biz/pizzeria-paradiso-geo...,862,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 38.9049770854945, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '3282 M St NW', 'address2': '', '...",+12023371245,(202) 337-1245,984.733365
1,3YAqTdLsvsApbJx1BvLFBg,curry-and-pie-washington,Curry & Pie,https://s3-media4.fl.yelpcdn.com/bphoto/2_RPlU...,False,https://www.yelp.com/biz/curry-and-pie-washing...,456,"[{'alias': 'indpak', 'title': 'Indian'}, {'ali...",4.0,"{'latitude': 38.9053344726562, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '1204 34th St NW', 'address2': No...",+12023334369,(202) 333-4369,787.308092
2,L_uhLGLeil2AvA-I9Y4vGQ,aladdin-house-of-kabob-and-gyros-washington-2,Aladdin House of Kabob & Gyros,https://s3-media3.fl.yelpcdn.com/bphoto/TAK2_e...,False,https://www.yelp.com/biz/aladdin-house-of-kabo...,5,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 38.9185, 'longitude': -77.07034}","[delivery, pickup]",NaN,"{'address1': '2132 Wisconsin Ave NW', 'address...",+12029443933,(202) 944-3933,1116.197120
3,UDxtqF8hTIhR0PzuIjd76g,paisanos-pizza-washington-4,Paisano's,https://s3-media1.fl.yelpcdn.com/bphoto/SG57H7...,False,https://www.yelp.com/biz/paisanos-pizza-washin...,59,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 38.9152897739676, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '1815 Wisconsin Ave NW', 'address...",+12022986800,(202) 298-6800,982.819403
4,7IcB-Ozkr0Irw7wD9ZKVVw,georgetown-pizza-and-grill-washington,Georgetown pizza & grill,https://s3-media2.fl.yelpcdn.com/bphoto/9Pjd69...,False,https://www.yelp.com/biz/georgetown-pizza-and-...,19,"[{'alias': 'pizza', 'title': 'Pizza'}]",3.5,"{'latitude': 38.90766, 'longitude': -77.06392}","[delivery, pickup]",NaN,"{'address1': '3208 O St NW', 'address2': None,...",+12023330333,(202) 333-0333,1021.951367


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             32 non-null     object 
 1   alias          32 non-null     object 
 2   name           32 non-null     object 
 3   image_url      32 non-null     object 
 4   is_closed      32 non-null     bool   
 5   url            32 non-null     object 
 6   review_count   32 non-null     int64  
 7   categories     32 non-null     object 
 8   rating         32 non-null     float64
 9   coordinates    32 non-null     object 
 10  transactions   32 non-null     object 
 11  price          28 non-null     object 
 12  location       32 non-null     object 
 13  phone          32 non-null     object 
 14  display_phone  32 non-null     object 
 15  distance       32 non-null     float64
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 3.9+ KB


In [26]:
##convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json', '.csv.gz')